In [44]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plot
import numpy as np
from numpy import genfromtxt
import csv
import sys, os, re
import pathlib

print(tf.__version__)

2.0.0


In [45]:
# The CANlab/WAVi Pain Study is Pre-Formatted to resemble BIDS neuroimaging formatting
# If your study does not abide to the following structure, please revisit previous scripts
# .../StudyRepo
# -------------> /raw
# -------------------> /*.art
# -------------------> /*.eeg
# -------------------> /*.evt
# -------------> /contigs
# -------------------> /train          2:1 train:test
# -------------------> /test

In [46]:
directory = input("Please give the full path of the contigs folder: ")
os.chdir(directory)

Please give the full path of the contigs folder: /home/clayton/science/CANlab/WAViMedEEG/CANlabStudy/contigs


In [47]:
train_path = pathlib.Path('train/')
test_path = pathlib.Path('test/')

In [48]:
# load in image paths for training set

import random
train_image_paths = list(train_path.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)

train_count = len(train_image_paths)
print("You have", train_count, "training images.")

# load in image paths for testing set?

test_image_paths = list(test_path.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
random.shuffle(test_image_paths)

test_count = len(test_image_paths)
print("You have", test_count, "testing images.")

You have 879 training images.
You have 440 testing images.


In [49]:
#list the available labels
label_names = sorted(item.name for item in train_path.glob('*/') if item.is_dir())
print("Labels discovered:", label_names)

Labels discovered: ['chronic', 'p300']


In [50]:
#assign an index to each label
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print("Label indices:", label_to_index)

Label indices: {'chronic': 0, 'p300': 1}


In [51]:
#create a list of every file and its index label
train_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                     for path in train_image_paths]

test_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in test_image_paths]

In [52]:
def load_numpy_stack(paths):
    numpy_stack = []
    for path in paths:
        numpy_stack.append(genfromtxt(path, delimiter=","))
    return(numpy_stack)

In [53]:
train_arrays = load_numpy_stack(train_image_paths)
test_arrays = load_numpy_stack(test_image_paths)

In [54]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_arrays, train_image_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_arrays, test_image_labels))

In [55]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [60]:
def kerasModel(learn):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(250, 19)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learn),
                 loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])
    return(model)

In [61]:
def fitModel(model):
    model.fit(train_dataset, epochs=10)
    return(model)

In [62]:
def testModel(model):
    metrics = model.evaluate(test_dataset)
    return(metrics)

In [74]:
# doing well with default 0.01
rate = float(input("What's my learning rate? "))

compiled = kerasModel(rate)
fitted = fitModel(compiled)
results = testModel(fitted)

What's my learning rate? 0.01
Epoch 1/10
14/14 [==============================] - 1s 38ms/step - loss: 88.0877 - sparse_categorical_accuracy: 0.4755
Epoch 2/10
14/14 [==============================] - 0s 9ms/step - loss: 41.4135 - sparse_categorical_accuracy: 0.6997
Epoch 3/10
14/14 [==============================] - 0s 9ms/step - loss: 17.1064 - sparse_categorical_accuracy: 0.7429
Epoch 4/10
14/14 [==============================] - 0s 9ms/step - loss: 9.3843 - sparse_categorical_accuracy: 0.8214
Epoch 5/10
14/14 [==============================] - 0s 8ms/step - loss: 5.5851 - sparse_categorical_accuracy: 0.8578
Epoch 6/10
14/14 [==============================] - 0s 9ms/step - loss: 3.2579 - sparse_categorical_accuracy: 0.8987
Epoch 7/10
14/14 [==============================] - 0s 8ms/step - loss: 4.5548 - sparse_categorical_accuracy: 0.8919
Epoch 8/10
14/14 [==============================] - 0s 9ms/step - loss: 2.0626 - sparse_categorical_accuracy: 0.9340
Epoch 9/10
14/14 [============

In [77]:
print("Loss: ", results[0], "\nAccuracy: ", results[1])

Loss:  24.97460147312709 
Accuracy:  0.6090909


In [69]:
rates

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])